## Подготовка данных для обучения модели CatBoost

Установить библиотеку catboost можно в Anaconda Prompt c помощью команды:

```
pip install catboost
```

Кроме того, если вы хотить строить графики обучения, вы должны установить пакет `ipywidgets` и  запустить специальную команду перед запуском тетрадки Jupiter:

```
$ pip install ipywidgets
$ jupyter nbextension enable --py widgetsnbextension
```

Импортируем необходимые библиотеки и набор данных.

In [1]:
from catboost.datasets import titanic
import numpy as np
import pandas as pd
train_df, test_df = titanic()  
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Прежде всего выясним, сколько у нас пропусков по каждой переменной.

In [2]:
train_df.isnull().sum()  

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Как видим, у переменных `Age`, `Cabin` и `Embarked` действительно есть пропуски, поэтому заполним их  некоторым числом вне диапазона распределения переменной (это эффективно для древовидных алгоритмов).

In [3]:
train_df.fillna(-999, inplace=True)  
test_df.fillna(-999, inplace=True)

Теперь выделим массив признаков и массив меток.

In [4]:
X = train_df.drop('Survived', axis=1)  
y = train_df.Survived

Зависимая переменная должна иметь тип `int`.
Обратите внимание, что наши признаки имеют разные типы – некоторые из них являются числовыми,
некоторые – категориальными, а некоторые – даже строками, которые обычно должны  обрабатываться определенным образом (например, с помощью представления bag-of-words). Однако  в нашем случае мы могли бы рассматривать эти строковые признаки как категориальные – вся  непростая работа будет выполнена внутри CatBoost.

In [5]:
print(X.dtypes)

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


Записываем индексы категориальных признаков.

In [6]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]  
categorical_features_indices

array([ 0,  1,  2,  3,  5,  6,  7,  9, 10])

Давайте разобьем наши обучающие данные на обучающий и контрольный наборы.

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.25, random_state=42)  
X_test = test_df

Для решения задачи классификации и задачи регрессии с помощью CatBoost в питоновской  библиотеке `scikit-learn` необходимо воспользоваться классами `CatBoostClassifier` и  `CatBoostRegressor` соответственно, предварительно импортировав их из библиотеки catboost.
У нас решается задача классификации, поэтому давайте импортируем класс `CatBoostClassifier`, а также классы `cv`, `Pool`, которыми мы воспользуемся позднее. Также импортируем функцию `accuracy_score()`.

In [8]:
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score

## Обучение модели CatBoost

Теперь давайте создадим саму модель: здесь мы можем задать значения параметров и гиперпараметров. По умолчанию для задачи классификации оптимизируется логистическая функция потерь (`loss_function='Logloss'`). C помощью параметра `eval_metric` мы можем оптимизировать нашу модель с точки зрения конкретной метрики (например, с точки зрения Accuracy: `eval_metric='Accuracy'`), для этого с помощью параметра `eval_set` нужно указать контрольный массив признаков и контрольный массив меток, на которых будет оптимизироваться метрика. Кроме того, с помощью параметра `custom_metric` можно справочно посмотреть, как обучается модель с точки зрения конкретной метрики. Например, мы оптимизируем нашу модель по Accuracy, но неплохо было бы посмотреть, как меняется F1-мера в зависимости от темпа обучения и количества итераций (`custom_metric='F1'`).

In [9]:
model = CatBoostClassifier(     
    loss_function='Logloss',
    eval_metric='Accuracy',
    custom_metric='F1',      
    random_seed=42,  
    logging_level='Silent'
)

In [10]:
model.fit(
    X_train, y_train,
    cat_features=categorical_features_indices,
    eval_set=(X_validation, y_validation),
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [11]:
print("Правильность на контрольной выборке: {:.2f}".format(model.score(X_validation, y_validation)))

Правильность на контрольной выборке: 0.83


## Перекрестная проверка в CatBoost

Проверка модели - хорошо, а перекрестная проверка – еще лучше. Воспользуемся классом `cv`. По  умолчанию он выполняет 3-блочную перекрестную проверку и возвращает результаты проверки в  виде объекта `DataFrame` библиотеки pandas, где для каждой итерации приводятся:
- значение оптимизируемой метрики, усредненное по контрольным блокам;
- стандартное отклонение оптимизируемой метрики, вычисленное по контрольным блокам;
- значение оптимизируемой метрики, усредненное по обучающим блокам;
- стандартное отклонение оптимизируемой метрики, вычисленное по обучающим блокам;
- значение функции потерь, усредненное по контрольным блокам;
- стандартное отклонение функции потерь, вычисленное по контрольным блокам;
- значение функции потерь, усредненное по обучающим блокам;
- стандартное отклонение функции потерь, вычисленное по обучающим блокам.

In [12]:
cv_params = model.get_params()  
cv_params.update({
    'loss_function': 'Logloss'
})
cv_data = cv(
    Pool(X, y, cat_features=categorical_features_indices),
    cv_params,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Теперь у нас для каждой итерации бустинга есть значения функции потерь и правильности, усредненные по 3  контрольным блокам перекрестной проверки, что должно дать нам более точную оценку качества нашей  модели.


In [13]:
print("Наилучшее значение правильности перекрестной проверки: {:.2f}±{:.2f} на шаге {}".format(
    np.max(cv_data['test-Accuracy-mean']),
    cv_data['test-Accuracy-std'][cv_data['test-Accuracy-mean'].idxmax()],
    cv_data['test-Accuracy-mean'].idxmax())
)

Наилучшее значение правильности перекрестной проверки: 0.82±0.02 на шаге 586


Как видим, наша первоначальная оценка качества при однократном разбиении была слишком оптимистичной – вот почему кросс-валидация так важна!

In [14]:
print("Точечное значение правильности перекрестной проверки: {}".format(np.max(cv_data['test-Accuracy-mean'])))

Точечное значение правильности перекрестной проверки: 0.819304152637486


## Обучение модели CatBoost с помощью класса Pool

Теперь для обучения модели воспользуемся классом `Pool`. В нем  хранится вся информация о наборе данных (признаки, метки, индексы категориальных признаков, веса и  многое другое).

Задаем значения параметров и гиперпараметров.

In [15]:
params = {
    'loss_function': 'Logloss',
    'eval_metric': 'Accuracy',
    'custom_metric': 'F1',
    'random_seed': 42,
    'logging_level': 'Silent'
}


Создаем обучающий и контрольный пулы.

In [16]:
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_validation, y_validation, cat_features=categorical_features_indices)

Обучаем модель.

In [17]:
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

## Выбор наилучшей модели CatBoost

Если у вас есть контрольная выборка, всегда лучше использовать гиперпараметр `use_best_model` во время  обучения. По умолчанию этот гиперпараметр включен, если с помощью параметра `eval_set` задан контрольный пул (контрольный массив признаков и контрольный массив меток). Если он включен, из ансамбля будет возвращена  итерация, дающая наилучшее значение Accuracy на контрольной выборке (ранее мы с помощью параметра  `eval_metric` задали оптимизируемую метрику, а с помощью параметра `eval_set` задали контрольный  массив признаков и массив меток).

In [18]:
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)

simple_model_params = params.copy()
simple_model_params.update({
    'use_best_model': False
})
simple_model = CatBoostClassifier(**simple_model_params)
simple_model.fit(train_pool, eval_set=validate_pool);

print('Правильность наилучшей модели на контрольной выборке: {:.3}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))
print('Правильность обычной модели на контрольной выборке: {:.3}'.format(
    accuracy_score(y_validation, simple_model.predict(X_validation))
))

Правильность наилучшей модели на контрольной выборке: 0.834
Правильность обычной модели на контрольной выборке: 0.821


Количество деревьев в наилучшей модели определяется следующим образом:
1. Строим количество деревьев в соответствии с выбранными параметрами и гиперпараметрами обучения.
2. Используем контрольную выборку для поиска итерации, на которой достигается оптимальное значение метрики, заданной с помощью параметра `eval_metric`.
3. Возвращаем модель на итерации, дающем оптимальное значение метрики, при этом деревья, построенные уже после найденной итерации, не будут сохранены. 
Давайте взглянем на количество деревьев в обычной и наилучшей моделях.

In [19]:
print(model.tree_count_)
print(simple_model.tree_count_)

158
1000


## Использование ранней остановки в CatBoost

Если у вас есть контрольная выборка, всегда проще и лучше использовать раннюю остановку. Эта  настройка похожа на предыдущую, но помимо улучшения качества она еще позволяет экономить  время. Раннюю остановку мы задаем с помощью детектора переобучения. Он настраивается с  помощью гиперпараметров `od_type` и `od_wait`.

In [20]:
%%time
simple_model = CatBoostClassifier(**simple_model_params)
simple_model.fit(train_pool, eval_set=validate_pool)

CPU times: user 1min 30s, sys: 14.7 s, total: 1min 45s
Wall time: 13.2 s


In [21]:
%%time
earlystop_params = params.copy()
earlystop_params.update({
    'od_type': 'Iter',
    'od_wait': 40
})
earlystop_model = CatBoostClassifier(**earlystop_params)
earlystop_model.fit(train_pool, eval_set=validate_pool)

CPU times: user 6.69 s, sys: 1.51 s, total: 8.2 s
Wall time: 1.05 s


In [22]:
print('Количество деревьев в обычной модели: {}'.format(simple_model.tree_count_))
print('Правильность обычной модели на контрольной выборке: {:.3}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))
print('')

print('Количество деревьев в модели с ранней остановкой: {}'.format(earlystop_model.tree_count_))
print('Правильность модели с ранней остановкой на контрольной выборке: {:.3}'.format(
    accuracy_score(y_validation, earlystop_model.predict(X_validation))
))

Количество деревьев в обычной модели: 1000
Правильность обычной модели на контрольной выборке: 0.834

Количество деревьев в модели с ранней остановкой: 36
Правильность модели с ранней остановкой на контрольной выборке: 0.83


Таким образом, мы получаем то же самое качество за более короткое время. Хотя, как было показано ранее, простая схема проверки не позволяет точно судить о качестве модели на новых данных (оценка качества может быть смещенной в силу того или иного разбиения данных), полезно отслеживать динамику улучшения модели и тем самым, как мы видим из этого примера, лучше остановить процесс обучения раньше (прежде чем мы начнем переобучаться).

## Использование базовой модели в CatBoost

Можно использовать предварительные (базовые) результаты обучения.

In [23]:
current_params = params.copy()
current_params.update({
    'iterations': 10
})
model = CatBoostClassifier(**current_params).fit(X_train, y_train, categorical_features_indices)
# получаем базовые результаты (только для prediction_type='RawFormulaVal')
baseline = model.predict(X_train, prediction_type = 'RawFormulaVal')
# обучаем новую модель
model.fit(X_train, y_train, categorical_features_indices, baseline=baseline)

## Поддержка контрольных точек в CatBoost

Catboost поддерживает контрольные точки. Вы можете применять их для возобновления обучения  или начала обучения, используя результаты предыдущей модели.

In [24]:
params_with_snapshot  = params.copy()
params_with_snapshot.update({
    'iterations': 5,
    'learning_rate': 0.5,
    'logging_level': 'Verbose'
})
model = CatBoostClassifier(**params_with_snapshot).fit(
    train_pool, eval_set=validate_pool, save_snapshot=True)
params_with_snapshot.update({
    'iterations': 20,
    'learning_rate': 0.1,
})
model = CatBoostClassifier(**params_with_snapshot).fit(
    train_pool, eval_set=validate_pool, save_snapshot=True)

0:	learn: 0.7919162	test: 0.7847534	best: 0.7847534 (0)	total: 18.4ms	remaining: 73.6ms
1:	learn: 0.8278443	test: 0.8206278	best: 0.8206278 (1)	total: 39.6ms	remaining: 59.4ms
2:	learn: 0.8293413	test: 0.8206278	best: 0.8206278 (1)	total: 53.1ms	remaining: 35.4ms
3:	learn: 0.8338323	test: 0.8206278	best: 0.8206278 (1)	total: 63.9ms	remaining: 16ms
4:	learn: 0.8368263	test: 0.8161435	best: 0.8206278 (1)	total: 72.9ms	remaining: 0us

bestTest = 0.8206278027
bestIteration = 1

Shrink model to first 2 iterations.
5:	learn: 0.8383234	test: 0.8161435	best: 0.8206278 (1)	total: 80.2ms	remaining: 101ms
6:	learn: 0.8398204	test: 0.8161435	best: 0.8206278 (1)	total: 89.3ms	remaining: 106ms
7:	learn: 0.8398204	test: 0.8161435	best: 0.8206278 (1)	total: 97.6ms	remaining: 98.8ms
8:	learn: 0.8458084	test: 0.8161435	best: 0.8206278 (1)	total: 105ms	remaining: 87.7ms
9:	learn: 0.8443114	test: 0.8161435	best: 0.8206278 (1)	total: 113ms	remaining: 79.8ms
10:	learn: 0.8473054	test: 0.8251121	best: 0.8251

## Важности признаков

Иногда очень важно понять, какой признак внес наибольший вклад в конечный результат. Для этого у модели CatBoost есть метод `.get_feature_importance()`.

In [26]:
model = CatBoostClassifier(iterations=50, random_seed=42, logging_level='Silent').fit(train_pool)
feature_importances = model.get_feature_importance(train_pool, fstr_type='FeatureImportance')
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

Sex: 31.702614119956866
Pclass: 20.207001310838848
Ticket: 11.973540755448495
Fare: 8.985295393916426
Cabin: 7.145337338432442
SibSp: 6.46811647401467
Parch: 4.987796626323452
Age: 4.718576113910789
Embarked: 3.8117218671580106
PassengerId: 0.0
Name: 0.0


Вывод показывает, что признаки `Sex` и `Pclass` имеют наибольшее влияние на результат.

## Получение прогнозов

Теперь получим прогнозы для первых 10 наблюдений.

In [27]:
predictions = model.predict(X_test)
predictions_probs = model.predict_proba(X_test)
print(predictions[:10])
print(predictions_probs[:10])

[0. 0. 0. 0. 1. 0. 1. 0. 1. 0.]
[[0.87564216 0.12435784]
 [0.79373093 0.20626907]
 [0.83797823 0.16202177]
 [0.89121482 0.10878518]
 [0.11485621 0.88514379]
 [0.89213536 0.10786464]
 [0.3672422  0.6327578 ]
 [0.7594323  0.2405677 ]
 [0.3932486  0.6067514 ]
 [0.951101   0.048899  ]]


## Сохранение модели

Наконец, сохраним построенную модель.

In [28]:
model = CatBoostClassifier(iterations=10, random_seed=42, logging_level='Silent').fit(train_pool)
model.save_model('catboost_model.dump')
model = CatBoostClassifier()
model.load_model('catboost_model.dump')

In [29]:
model.save_model(
    'model.json',
    format='json',
    # pool=pool  # this parameter is required only for models with categorical features.
)

In [30]:
import json
model = json.load(open('model.json', 'r'))
model.keys()

dict_keys(['oblivious_trees', 'ctr_data', 'model_info', 'features_info'])

In [32]:
model['features_info'].keys()

dict_keys(['categorical_features', 'ctrs', 'float_features'])

In [33]:
model['features_info']['float_features'][0]

{'nan_value_treatment': 'AsIs',
 'has_nans': False,
 'flat_feature_index': 4,
 'feature_index': 0,
 'borders': [24.25, 32.25, 39.5]}

In [46]:
model['features_info']['ctrs'][0]

{'borders': [4.999999046, 10.99999905],
 'prior_numerator': 0,
 'shift': 0,
 'target_border_idx': 0,
 'ctr_type': 'Borders',
 'scale': 15,
 'elements': [{'cat_feature_index': 1,
   'combination_element': 'cat_feature_value'}],
 'identifier': '{"identifier":[{"cat_feature_index":1,"combination_element":"cat_feature_value"}],"type":"Borders"}',
 'prior_denomerator': 1}

In [47]:
model['features_info']['categorical_features'][0]

{'flat_feature_index': 0, 'feature_index': 0}

In [34]:
def dump_json(item):
    print(json.dumps(item, indent=2))

dump_json(model['oblivious_trees'][0])  # первое дерево

{
  "leaf_weights": [
    3,
    6,
    2,
    3,
    89,
    48,
    3,
    35,
    1,
    0,
    1,
    0,
    80,
    0,
    50,
    0,
    0,
    5,
    0,
    0,
    0,
    342,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0
  ],
  "leaf_values": [
    -0.3029632365,
    -0.3383094409,
    -0.8565402698,
    -0.3029632365,
    0.05623899168,
    -0.3942285282,
    0.3029632365,
    -0.390077816,
    0.5634371199,
    0,
    0.5634371199,
    0,
    0.9729528723,
    0,
    1.580343992,
    0,
    0,
    -0.6415095758,
    0,
    0,
    0,
    -0.8480673873,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0
  ],
  "splits": [
    {
      "split_index": 6,
      "cat_feature_index": 3,
      "value": 2083200611,
      "split_type": "OneHotFeature"
    },
    {
      "split_index": 4,
      "float_feature_index": 1,
      "split_type": "FloatFeature",
      "border": 57.48960114
    },
    {
      "split_index": 17,
      "split_type": "Online

In [38]:
split_list = []
for float_feature in model['features_info']['float_features']:
    if not float_feature['borders']:
        continue
    for border in float_feature['borders']:
        split_list.append(
            {
                'split_index': len(split_list),
                'float_feature_index': float_feature['feature_index'], 
                'border_id': border, 
                'split_type': 'FloatFeature',
                'flat_feature_index': float_feature['flat_feature_index']
            }
        )

In [39]:
first_tree = model['oblivious_trees'][0]
first_tree['splits']

[{'split_index': 6,
  'cat_feature_index': 3,
  'value': 2083200611,
  'split_type': 'OneHotFeature'},
 {'split_index': 4,
  'float_feature_index': 1,
  'split_type': 'FloatFeature',
  'border': 57.48960114},
 {'split_index': 17,
  'split_type': 'OnlineCtr',
  'border': 3.999999046,
  'ctr_target_border_idx': 0},
 {'split_index': 12,
  'split_type': 'OnlineCtr',
  'border': 8.999999046,
  'ctr_target_border_idx': 0},
 {'split_index': 24,
  'split_type': 'OnlineCtr',
  'border': 10.99999905,
  'ctr_target_border_idx': 0}]